In [1]:
%load_ext lab_black

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm
from types import SimpleNamespace
from bnnrul.cmapss.preprocessing import generate_parquet, generate_lmdb

/tmp/ipykernel_2581429/741591142.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
args = SimpleNamespace(
    out_path="../data/cmapss/",
    normalization="min-max",
    validation=0.2,
    subsets=["FD001"],  # "FD002", "FD003", "FD004"],
    # win_length={"FD001": 30, "FD002": 20, "FD003": 30, "FD004": 15}, #variable length per subset only ok for LSTM models
    win_length={
        1: 18,
        2: 18,
        3: 18,
        4: 18,
    },  # variable length per subset fixed for Linear/Conv models
    win_step=1,
    settings=["setting_1", "setting_2", "setting_3"],
    features=[
        "sensor_2",
        "sensor_3",
        "sensor_4",
        "sensor_7",
        "sensor_8",
        "sensor_9",
        "sensor_11",
        "sensor_12",
        "sensor_13",
        "sensor_14",
        "sensor_15",
        "sensor_17",
        "sensor_20",
        "sensor_21",
    ],
    bits=32,
)

In [6]:
generate_parquet(args)

Extracting dataframes...
Number of training trajectories = 80
Number of validation trajectories = 20
Done.
Generating parquet files...


In [6]:
df_train = pd.read_parquet(f"{args.out_path}/parquet/train_FD001.parquet")
print(df_train.groupby("trajectory_id").ngroups)
df_train.query("trajectory_id==2")

80


,trajectory_id,setting_1,setting_2,setting_3,sensor_2,sensor_3,sensor_4,sensor_7,sensor_8,sensor_9,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_17,sensor_20,sensor_21,rul
192,2,-0.0018,0.0006,100.0,641.89,1583.84,1391.28,554.53,2388.01,9054.72,46.93,522.33,2388.06,8137.72,8.3905,391,38.94,23.4585,286
193,2,0.0043,-0.0003,100.0,641.82,1587.05,1393.13,554.77,2387.98,9051.31,47.24,522.70,2387.98,8131.09,8.4167,392,39.06,23.4085,285
194,2,0.0018,0.0003,100.0,641.55,1588.32,1398.96,555.14,2388.04,9054.24,47.22,522.58,2387.99,8140.58,8.3802,391,39.11,23.4250,284
195,2,0.0035,-0.0004,100.0,641.68,1584.15,1396.08,554.25,2387.98,9058.01,47.10,522.49,2387.93,8140.44,8.4018,391,39.13,23.5027,283
196,2,0.0005,0.0004,100.0,641.73,1579.03,1402.52,555.12,2388.03,9058.15,47.25,522.27,2387.94,8136.67,8.3867,390,39.18,23.4234,282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,2,0.0046,0.0002,100.0,643.78,1602.03,1429.67,551.46,2388.16,9084.13,48.21,520.07,2388.20,8174.85,8.5199,398,38.42,23.0358,4
475,2,-0.0006,0.0001,100.0,643.91,1601.35,1430.04,551.96,2388.22,9089.87,48.18,519.95,2388.21,8166.83,8.5291,395,38.23,23.1196,3
476,2,-0.0007,0.0004,100.0,643.67,1596.84,1431.17,550.85,2388.20,9098.67,48.27,519.91,2388.22,8164.83,8.5242,396,38.39,23.1155,2
477,2,-0.0010,-0.0003,100.0,643.44,1603.63,1429.57,551.61,2388.18,9102.01,48.14,519.51,2388.22,8169.97,8.4932,395,38.33,23.0169,1


In [47]:
df_validation = pd.read_parquet(f"{args.out_path}/parquet/val_FD001.parquet")
(df_validation.groupby("trajectory_id").ngroups)
df_validation.query("trajectory_id==13")

,trajectory_id,sensor_2,sensor_3,sensor_4,sensor_7,sensor_8,sensor_9,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_17,sensor_20,sensor_21,rul
2546,13,641.94,1594.32,1405.50,554.14,2388.07,9051.55,47.46,520.87,2388.11,8131.99,8.4393,392,38.76,23.3793,162
2547,13,642.61,1592.31,1406.78,553.08,2388.11,9053.39,47.50,521.45,2388.08,8135.24,8.4325,393,38.97,23.2731,161
2548,13,642.23,1589.66,1410.36,553.29,2388.10,9061.71,47.43,521.66,2388.10,8137.34,8.4642,393,38.67,23.2775,160
2549,13,642.67,1593.12,1408.66,553.71,2388.12,9047.99,47.65,521.44,2388.16,8133.29,8.4115,391,38.80,23.3566,159
2550,13,642.77,1589.93,1408.31,553.21,2388.08,9057.89,47.56,521.32,2388.14,8133.30,8.3731,393,39.08,23.4748,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,13,643.53,1612.88,1418.86,551.15,2388.15,9146.79,48.11,520.50,2388.20,8212.93,8.5540,396,38.28,23.1357,4
2705,13,643.77,1597.42,1429.73,551.20,2388.17,9140.98,48.04,519.80,2388.13,8210.94,8.5109,397,38.49,23.2090,3
2706,13,643.50,1602.21,1432.17,551.85,2388.11,9144.17,48.34,520.26,2388.19,8210.25,8.5269,396,38.43,23.0185,2
2707,13,643.87,1605.83,1431.55,551.48,2388.16,9149.29,48.28,520.08,2388.15,8218.44,8.5015,397,38.35,23.1104,1


In [30]:
df_test = pd.read_parquet(f"{args.out_path}/parquet/test_FD001.parquet")
print(df_test.groupby("trajectory_id").ngroups)
df_test.query("trajectory_id==1")

100


,trajectory_id,sensor_2,sensor_3,sensor_4,sensor_7,sensor_8,sensor_9,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_17,sensor_20,sensor_21,rul
0,1,643.02,1585.29,1398.21,553.90,2388.04,9050.17,47.20,521.72,2388.03,8125.55,8.4052,392,38.86,23.3735,112
1,1,641.71,1588.45,1395.42,554.85,2388.01,9054.42,47.50,522.16,2388.06,8139.62,8.3803,393,39.02,23.3916,111
2,1,642.46,1586.94,1401.34,554.11,2388.05,9056.96,47.50,521.97,2388.03,8130.10,8.4441,393,39.08,23.4166,110
3,1,642.44,1584.12,1406.42,554.07,2388.03,9045.29,47.28,521.38,2388.05,8132.90,8.3917,391,39.00,23.3737,109
4,1,642.51,1587.19,1401.92,554.16,2388.01,9044.55,47.31,522.15,2388.03,8129.54,8.4031,390,38.99,23.4130,108
5,1,642.11,1579.12,1395.13,554.22,2388.00,9050.96,47.26,521.92,2388.08,8127.46,8.4238,392,38.91,23.3467,107
6,1,642.11,1583.34,1404.84,553.89,2388.05,9051.39,47.31,522.01,2388.06,8134.97,8.3914,391,38.85,23.3952,106
7,1,642.54,1580.89,1400.89,553.59,2388.05,9052.86,47.21,522.09,2388.06,8125.93,8.4213,393,39.05,23.3224,105
8,1,641.88,1593.29,1412.28,554.49,2388.06,9048.55,47.37,522.03,2388.05,8134.15,8.4353,391,39.10,23.4521,104
9,1,642.07,1585.25,1398.64,554.28,2388.04,9051.95,47.14,522.00,2388.06,8134.08,8.4093,391,38.87,23.3820,103


In [20]:
with zipfile.ZipFile(f"{args.out_path}/CMAPSSData.zip") as zip_file:
    file_rul = zip_file.open("RUL_FD001.txt")
np.asarray(file_rul.readlines(), dtype=np.int32)

array([112,  98,  69,  82,  91,  93,  91,  95, 111,  96,  97, 124,  95,
       107,  83,  84,  50,  28,  87,  16,  57, 111, 113,  20, 145, 119,
        66,  97,  90, 115,   8,  48, 106,   7,  11,  19,  21,  50, 142,
        28,  18,  10,  59, 109, 114,  47, 135,  92,  21,  79, 114,  29,
        26,  97, 137,  15, 103,  37, 114, 100,  21,  54,  72,  28, 128,
        14,  77,   8, 121,  94, 118,  50, 131, 126, 113,  10,  34, 107,
        63,  90,   8,   9, 137,  58, 118,  89, 116, 115, 136,  28,  38,
        20,  85,  55, 128, 137,  82,  59, 117,  20], dtype=int32)

In [7]:
generate_lmdb(args, datasets=["train"])

FileExistsError: [Errno 17] File exists: '../data/cmapss/lmdb'

In [36]:
import ast
import torch
from torch.utils.data import DataLoader
from mltbox.utils.lmdb_utils import LmdbDataset


class LmdbDatasetRul(LmdbDataset):
    def __getitem__(self, i: int):
        sample = super().__getitem__(i)
        rul = self.dtype(super().get(f"rul_{i}", numpy=False))
        return sample.copy(), rul


class LmdbDatasetAll(LmdbDatasetRul):
    def __getitem__(self, i: int):
        sample, rul = super().__getitem__(i)
        ds_id = int(super().get(f"ds_id_{i}", numpy=False))
        traj_id = int(super().get(f"traj_id_{i}", numpy=False))
        win_id = int(super().get(f"win_id_{i}", numpy=False))
        settings = super().get(f"settings_{i}", numpy=True)
        return ds_id, traj_id, win_id, settings.copy(), rul, sample


ds = LmdbDatasetRul(
    f"{args.out_path}/lmdb/train.lmdb", "{}", input_features=args.features
)

dl = DataLoader(
    ds,
    shuffle=False,
    batch_size=len(ds),
    num_workers=1,
    pin_memory=True,
)
dtype = torch.float32 if int(ds.get("bits", numpy=False)) == 32 else torch.float64
torch.set_default_dtype(dtype)
win_length, n_features = (
    ds.get("win_length", numpy=False),
    ds.n_features,
)
# if isinstance(win_length, dict):
win_length = ast.literal_eval(win_length)

In [32]:
for sample, rul in dl:
    print(sample.shape, rul)
    break

torch.Size([1, 18, 14]) tensor([269.])


In [34]:
for ds_id, traj_id, win_id, settings, rul, sample in dl:
    print(ds_id, traj_id, win_id, settings.shape, rul, sample.shape)
    break

float32
float32
float32

NameError: Caught NameError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/luis/miniconda3/envs/pt/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/luis/miniconda3/envs/pt/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/luis/miniconda3/envs/pt/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_2310580/964139755.py", line 22, in __getitem__
    aaa
NameError: name 'aaa' is not defined


In [12]:
win_length  # , n_features

{1: 18, 2: 18, 3: 18, 4: 18}

In [37]:
from models import Model
import pytorch_lightning as pl

input_size = (1, win_length[1], n_features)
model = Model(input_size)
trainer = pl.Trainer(
    gpus=[1],
    max_epochs=100,
    log_every_n_steps=100,
    logger=pl.loggers.TensorBoardLogger(".", "lightning_logs"),
    auto_scale_batch_size=True,
)
trainer.fit(model, dl)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name   | Type   | Params
----------------------------------
0 | layers | Linear | 45.6 K
----------------------------------
45.6 K    Trainable params
0         Non-trainable params
45.6 K    Total params
0.182     Total estimated model params size (MB)
/home/luis/miniconda3/envs/pt/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=100). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/home/luis/repos/bnnrul/models.py:56: UserWarning: Using a target size (torch.Size([15308])) that is different to the input size (torch.Size([15308, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y)
